In [ ]:
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def load_dataset(file_path):
    return pd.read_csv(file_path)

def handle_missing_data(df, feature_handling):
    for feature, details in feature_handling.items():
        if details['is_selected']:
            if 'missing_values' in details['feature_details']:
                if details['feature_details']['missing_values'] == 'Impute':
                    if details['feature_details']['impute_with'] == 'Average of values':
                        df[feature] = df[feature].fillna(df[feature].mean())
                    elif details['feature_details']['impute_with'] == 'custom':
                        df[feature] = df[feature].fillna(details['feature_details']['impute_value'])
                    
    return df

def feature_reduction(df, method, num_features_to_keep, prediction_type, target):
    if method == 'PCA':
        pca = PCA(n_components=num_features_to_keep)
        return pca.fit_transform(df)
    elif method == 'Tree-based':
        selector = SelectKBest(score_func=f_classif if prediction_type == 'classification' else f_regression, k=num_features_to_keep)
        return selector.fit_transform(df, target)
    else:
        raise ValueError(f"Unknown feature reduction method: {method}")
        
def get_model(config):
    model_name = None
    for alg_name, alg_details in config['algorithms'].items():
        if alg_details['is_selected']:
            model_name = alg_name
            break
    if model_name == 'RandomForestRegressor':
        return RandomForestRegressor()
    elif model_name == 'RandomForestClassifier':
        return RandomForestClassifier()
    else:
        raise ValueError(f"Unknown or unsupported model: {model_name}")

